# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.core import Model 
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
import json
import requests

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone'

experiment=Experiment(ws, experiment_name)

In [3]:
path = "https://raw.githubusercontent.com/ewellmann/capstone/master/starter_file/heart_failure_clinical_records_dataset.csv"
ds = TabularDatasetFactory.from_delimited_files(path)
data = ds.to_pandas_dataframe().dropna()
data.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [4]:
# Compute name should contain only letters, digits, hyphen and should be 2-16 charachters long
compute_name = "capstonecompute"


# Check if the compute cluster already exists, if not create a new one
if compute_name not in ws.compute_targets:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2S_V3",
                                                           min_nodes=0,
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)
    compute_target.wait_for_completion(show_output=True)
else:
    compute_target = ws.compute_targets[compute_name]


InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
        task='classification',
        compute_target=compute_target,
        training_data=ds,
        label_column_name='DEATH_EVENT',
        n_cross_validations=4,
        **automl_settings
)

In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
capstone,AutoML_ff80b5fc-37cf-498a-8f88-ea8e66f5e5d2,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
capstone,AutoML_ff80b5fc-37cf-498a-8f88-ea8e66f5e5d2,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

{'runId': 'AutoML_ff80b5fc-37cf-498a-8f88-ea8e66f5e5d2',
 'target': 'capstonecompute',
 'status': 'Completed',
 'startTimeUtc': '2023-08-24T07:10:13.161105Z',
 'endTimeUtc': '2023-08-24T07:28:30.810875Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'capstonecompute',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone","subscription_id":"a0a76bad-11a1-4a2d-9887-97a29122c8ed","resource_group":"aml-quickstarts-240614","workspace_name":"quick-starts-ws-240614","region":"westus2","compute_target":"capstonecompute","sp

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
# Get best run and model
best_run, fitted_model = remote_run.get_output()

# Print the best run
print(best_run)

# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()

# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

Run(Experiment: capstone,
Id: AutoML_ff80b5fc-37cf-498a-8f88-ea8e66f5e5d2_48,
Type: azureml.scriptrun,
Status: Completed)
precision_score_macro 0.8760186837149371
accuracy 0.8695045045045046
f1_score_macro 0.84073194542888
balanced_accuracy 0.8307690122573597
AUC_micro 0.915163452641831
f1_score_micro 0.8695045045045046
average_precision_score_micro 0.915562346113076
average_precision_score_macro 0.889976509508614
precision_score_weighted 0.8794294176424418
recall_score_macro 0.8307690122573597
recall_score_micro 0.8695045045045046
AUC_weighted 0.9085346418856305
log_loss 0.4220731707489711
weighted_accuracy 0.8966915362259028
f1_score_weighted 0.8632718468537706
matthews_correlation 0.7043307068823089
AUC_macro 0.9085346418856306
precision_score_micro 0.8695045045045046
average_precision_score_weighted 0.9148669164036838
norm_macro_recall 0.6615380245147194
recall_score_weighted 0.8695045045045046
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_ff80b5fc-37cf-498a-8f88-ea8e66

In [9]:
#TODO: Save the best model
best_model = best_run.register_model(model_path='outputs/model.pkl', model_name='capstone_model_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'Accuracy': best_run_metrics['accuracy']})

print(best_model)

Model(workspace=Workspace.create(name='quick-starts-ws-240614', subscription_id='a0a76bad-11a1-4a2d-9887-97a29122c8ed', resource_group='aml-quickstarts-240614'), name=capstone_model_automl, id=capstone_model_automl:1, version=1, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '0.8695045045045046'})


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [27]:
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'env.yml')

In [26]:

inference_config = InferenceConfig(entry_script='score.py', environment='env.yml')


deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
service = Model.deploy(workspace=ws, name='capstoneservice', models=[best_model], inference_config=inference_config, deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-08-24 10:41:41+00:00 Creating Container Registry if not exists.
2023-08-24 10:41:41+00:00 Generating deployment configuration.
2023-08-24 10:41:42+00:00 Submitting deployment to compute.
2023-08-24 10:41:49+00:00 Checking the status of deployment capstoneservice..
2023-08-24 10:43:53+00:00 Checking the status of inference endpoint capstoneservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [24]:

# test data
test_df = data.sample(5) 
label_df = test_df.pop('DEATH_EVENT')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)

# content type
headers = {'Content-type': 'application/json'}


response = requests.post(service.scoring_uri, test_sample, headers=headers)
# Print results from the inference
print(response.text)
# Print original labels
print(label_df)

{"data": [{"age": 60.0, "anaemia": 1, "creatinine_phosphokinase": 588, "diabetes": 1, "ejection_fraction": 60, "high_blood_pressure": 0, "platelets": 194000.0, "serum_creatinine": 1.1, "serum_sodium": 142, "sex": 0, "smoking": 0, "time": 33}, {"age": 54.0, "anaemia": 1, "creatinine_phosphokinase": 427, "diabetes": 0, "ejection_fraction": 70, "high_blood_pressure": 1, "platelets": 151000.0, "serum_creatinine": 9.0, "serum_sodium": 137, "sex": 0, "smoking": 0, "time": 196}, {"age": 75.0, "anaemia": 0, "creatinine_phosphokinase": 675, "diabetes": 1, "ejection_fraction": 60, "high_blood_pressure": 0, "platelets": 265000.0, "serum_creatinine": 1.4, "serum_sodium": 125, "sex": 0, "smoking": 0, "time": 205}, {"age": 65.0, "anaemia": 1, "creatinine_phosphokinase": 335, "diabetes": 0, "ejection_fraction": 35, "high_blood_pressure": 1, "platelets": 235000.0, "serum_creatinine": 0.8, "serum_sodium": 136, "sex": 0, "smoking": 0, "time": 120}, {"age": 65.0, "anaemia": 1, "creatinine_phosphokinase":

TODO: In the cell below, print the logs of the web service and delete the service

In [22]:
print(service.get_logs())

2023-08-24T08:06:53,007792800+00:00 - rsyslog/run 
2023-08-24T08:06:53,015584900+00:00 - gunicorn/run 
2023-08-24T08:06:53,026650400+00:00 | gunicorn/run | 
2023-08-24T08:06:53,028666100+00:00 | gunicorn/run | ###############################################
2023-08-24T08:06:53,030830200+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-08-24T08:06:53,032452700+00:00 | gunicorn/run | ###############################################
2023-08-24T08:06:53,034448300+00:00 | gunicorn/run | 
2023-08-24T08:06:53,060733400+00:00 | gunicorn/run | 
2023-08-24T08:06:53,087801600+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2
2023-08-24T08:06:53,087772900+00:00 - nginx/run 
2023-08-24T08:06:53,091275900+00:00 | gunicorn/run | 
2023-08-24T08:06:53,095139300+00:00 | gunicorn/run | 
2023-08-24T08:06:53,098717500+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

In [25]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
